# 4.8 Homework :  Deployment (Answers)
Panayotis Papoutsis

## Q1. Notebook

In [10]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [8]:
import pickle
import pandas as pd
import numpy as np
from datetime import datetime
import sklearn

In [44]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

    categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

/opt/homebrew/Caskroom/miniforge/base/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [14]:
year = 2021
month = 2
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [17]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

pred_mean = np.round(np.mean(y_pred),2)
print(f'The mean predicted for the February 2021 FVH dataset is {pred_mean} minutes.')


The mean predicted for the February 2021 FVH dataset is 16.19 minutes.


## Q2. Preparing the output

In [19]:
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 9.6 MB/s eta 0:00:00:00:0100:01


In [20]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

output_file = f'data/predictions_fhv_tripdata_{year:04d}-{month:02d}.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False,
    )

In [24]:
!du -sh data/*.parquet

 19M	data/predictions_fhv_tripdata_2021-02.parquet


## Q3. Creating the scoring script

The command to transform a jupyter notebook into a script is : jupyter nbconvert --to script homework-answer.ipynb

In [ ]:
!jupyter nbconvert --to script starter.ipynb

## Q4. Virtual environment

In [27]:
!pip install pipenv

  Using cached pipenv-2022.6.7-py2.py3-none-any.whl (3.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 11.7 MB/s eta 0:00:0000:010:01
  Using cached virtualenv_clone-0.5.7-py3-none-any.whl (6.6 kB)
  Using cached distlib-0.3.4-py2.py3-none-any.whl (461 kB)
  Using cached platformdirs-2.5.2-py3-none-any.whl (14 kB)
  Using cached filelock-3.7.1-py3-none-any.whl (10 kB)


In [31]:
!pipenv install scikit-learn==1.0.2 pyarrow pandas numpy mlflow prefect==2.0b6

Installing scikit-learn==1.0.2...
Adding scikit-learn to Pipfile's [packages]...
✔ Installation Succeeded 
Installing pyarrow...
Adding pyarrow to Pipfile's [packages]...
✔ Installation Succeeded 
Installing pandas...
Adding pandas to Pipfile's [packages]...
✔ Installation Succeeded 
Installing numpy...
Adding numpy to Pipfile's [packages]...
✔ Installation Succeeded 
Installing mlflow...
Adding mlflow to Pipfile's [packages]...
✔ Installation Succeeded 
Installing prefect==2.0b6...
Adding prefect to Pipfile's [packages]...
✔ Installation Succeeded 
Installing dependencies from Pipfile.lock (b41608)...
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 2/2 — 00:00:00:00:00:00:00
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [32]:
!cat Pipfile.lock | grep -2 scikit-learn

            "version": "==0.6.0"
        },
        "scikit-learn": {
            "hashes": [
                "sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b",


## Q5. Parametrize the script

In [34]:
!python starter.py 2021 3

16.3


## Q6. Docker container

In [37]:
!cat docker_starter.py

from flask import Flask, request, jsonify
import numpy as np
import pickle
import pandas as pd
import numpy as np



def load_dic(file):
    with open(file, 'rb') as f_in:
        dv, lr = pickle.load(f_in)
    return dv,lr
    
def read_data(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    return df

def data_processing(year,month):

    df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')
    categorical = ['PUlocationID', 'DOlocationID']
    dv, lr = load_dic('model.bin')
    dicts = df[categorical].to_dict(orient='records')

    X_val = dv.transform(dicts)
    return lr,X_val

def predict(lr,X_val):
    y_pred = lr.predict(X_val)
    #pred_mean = np.round(np.mean(y_pred),2)
    return y_pred


app = Flask('homework-week4')    

@app.route('/predict', 

In [36]:
!cat test.py


import requests

date = {"year" : 2021,
        "month": 4}

url = 'http://localhost:9696/predict'
rep = requests.post(url,json = date)

print(rep.json())

The flask application is tested with test.py .

In [38]:
!python test.py

{'mean prediction': 21.58}


We build the image with this command : `docker build -t mean-duration-service:v1 .`. Next, we launch the docker image with `docker run -it --rm -p 9696:9696 mean-duration-service:v1`. 

In [43]:
!python test.py

{'mean prediction': 21.58}


The closest answer is `25.96`, not `16.55` as in my answer in the google form. 

##  Bonus: upload the result to the cloud (Not graded)

By replacing the line `output_file =  f'predictions_fhv_tripdata_{year:04d}-{month:02d}.parquet'` with `output_file = f's3://mlflow-artifacts-remote-store/homework-week4/predictions_fhv_tripdata_{year:04d}-{month:02d}.parquet'`in the docker_starter.py . The prerequisite is to run the script from the EC2 instance that is associated with the S3 bucket `mlflow-artifacts-remote-store`.